In [1]:
from pynytimes import NYTAPI
import datetime
import pandas as pd

import numpy as np

In [2]:
api_key = "uLZTQY03vZfRZfk6nmzKcSc82EEUj4lK"

In [3]:
def get_news(year, month, day, nyt=None, k=10):
    """
    get top k most relevent finance news headings on each day from NY times
    """
    res = []
    try:
        if not nyt:
            nyt = NYTAPI(api_key, parse_dates=True)

        articles = nyt.article_search(
                results = k,
                dates = {
                    "begin": datetime.datetime(year, month, day),
                    "end": datetime.datetime(year, month, day)
                },
                options = {
                    "sort": "relevance",
                    "news_desk": [
                        "Business", "Business Day", "Entrepreneurs", "Financial", "Technology"
                    ],
                    "section_name" : [
                        "Business", "Business Day", "Technology"
                    ]
                }
            )

        for i in range(len(articles)):
            timestamp = articles[i]['pub_date']
            article_url = articles[i]['web_url']
            lead_paragraph = articles[i]['lead_paragraph']
            abstract = articles[i]['abstract']
            res.append({'timestamp': timestamp,
                        'article_url': article_url,
                        'lead_paragraph': lead_paragraph,
                        'abstract': abstract,
                        })
        
    except Exception as e:
        res.append({})
    
    return res


def generate_news(dates):
    """_summary_

    Parameters
    ----------
    dates : pd.Series 
        Series of Datetime objects.
    """
    nyt = NYTAPI(api_key, parse_dates=True)
    data = []
    for i, date in enumerate(dates):
        print(i)
        y, m, d = date.year, date.month, date.day
        news_list = get_news(y, m, d, nyt=nyt)
        data.extend(news_list)
            
    df = pd.DataFrame(data=data)
    return df

#### Get dates from our SPY data.

In [4]:
spy_df = pd.read_csv('../data/spy_prices.csv')
dates = pd.to_datetime(spy_df['Date'], utc=True)

len(dates)

1258

### Pull NYT data.
 - There's a 500 request limit per day and 5 request limit per min.

In [10]:
res_df = generate_news(dates[-280:])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [11]:
res_df.head()

,timestamp,article_url,lead_paragraph,abstract
0,2023-08-18 16:14:19+00:00,https://www.nytimes.com/interactive/2023/08/18...,New funding has helped the I.R.S. increase sta...,New funding has helped the I.R.S. increase sta...
1,2023-08-18 11:55:07+00:00,https://www.nytimes.com/2023/08/18/business/in...,"In the span of a month, the bottom has dropped...","Apple, Nvidia, Tesla and Microsoft are among t..."
2,2023-08-18 09:00:02+00:00,https://www.nytimes.com/interactive/2023/08/18...,The concept that China has reached the pinnacl...,The concept that China has reached the pinnacl...
3,2023-08-18 08:06:58+00:00,https://www.nytimes.com/2023/08/18/business/ho...,Stocks in Hong Kong entered a bear market on F...,The Hang Seng Index has fallen more than 20 pe...
4,2023-08-18 04:01:21+00:00,https://www.nytimes.com/2023/08/18/business/in...,He is known as the Minister for Everything. Fr...,The fate of Indonesia’s unrivaled stocks of ni...


#### Check if the news falls before or after market closes on each day. If it's after close, then the news should be categorised as a next day news.

In [12]:
threshold_time = pd.Timestamp('16:00:00').time()

res_df['adjusted_date'] = res_df['timestamp'].apply(lambda x: x.date() if x.time() < threshold_time else (x + pd.Timedelta(days=1)).date())
res_df.head()

,timestamp,article_url,lead_paragraph,abstract,adjusted_date
0,2023-08-18 16:14:19+00:00,https://www.nytimes.com/interactive/2023/08/18...,New funding has helped the I.R.S. increase sta...,New funding has helped the I.R.S. increase sta...,2023-08-19
1,2023-08-18 11:55:07+00:00,https://www.nytimes.com/2023/08/18/business/in...,"In the span of a month, the bottom has dropped...","Apple, Nvidia, Tesla and Microsoft are among t...",2023-08-18
2,2023-08-18 09:00:02+00:00,https://www.nytimes.com/interactive/2023/08/18...,The concept that China has reached the pinnacl...,The concept that China has reached the pinnacl...,2023-08-18
3,2023-08-18 08:06:58+00:00,https://www.nytimes.com/2023/08/18/business/ho...,Stocks in Hong Kong entered a bear market on F...,The Hang Seng Index has fallen more than 20 pe...,2023-08-18
4,2023-08-18 04:01:21+00:00,https://www.nytimes.com/2023/08/18/business/in...,He is known as the Minister for Everything. Fr...,The fate of Indonesia’s unrivaled stocks of ni...,2023-08-18


In [13]:
res_df.to_csv("../data/nyt_headlines.csv", index=False)